In [16]:
import pandas as pd

# Load the datasets
tracks_df = pd.read_csv('Data/Tracks/20_tracks.csv')
book_df = pd.read_csv('Data/Tracks/Book20.csv')

# Preprocess track IDs in Book29.csv to handle multiple IDs in a single cell
expanded_rows = []
for index, row in book_df.iterrows():
    track_ids = str(row['trackId']).split(', ')
    for track_id in track_ids:
        expanded_rows.append({
            'trackId': int(track_id.strip()),
            'initial_frame': row['initial_frame'],
            'ending_frame': row['ending_frame']
        })

# Convert the expanded list into a DataFrame
book_expanded_df = pd.DataFrame(expanded_rows)

# Create a mapping from the book dataframe
range_mapping = {}
for _, row in book_expanded_df.iterrows():
    if row['trackId'] not in range_mapping:
        range_mapping[row['trackId']] = []
    range_mapping[row['trackId']].append((row['initial_frame'], row['ending_frame']))

# Label the tracks DataFrame
def label_stopped(row):
    if row['trackId'] in range_mapping:
        for frame_range in range_mapping[row['trackId']]:
            if frame_range[0] <= row['frame'] <= frame_range[1]:
                return 1
    return 0

tracks_df['stopped_before_pedestrian'] = tracks_df.apply(label_stopped, axis=1)

# Save the updated dataframe
tracks_df.to_csv('Data/20_tracks_labeled.csv', index=False)
